In [ ]:
from copy import deepcopy

import polars as pd
import plotly.express as px
import pw_mapping

In [ ]:
pages = pw_mapping.Page.select()
df_pages = pd.DataFrame([{"page_id": page.page_id, "title": page.title, "item_id": page.item_id, "views": page.views} for page in pages])
df_pages

In [ ]:
t_df = df_pages.filter(df_pages["views"] < 200)
px.histogram(t_df, x="views", nbins=200)

In [ ]:
cursor = pw_mapping.database.execute_sql("select item_id, count(item_id) from item_aliases group by item_id")
df_aliases = pd.DataFrame([{"item_id": alias[0], "count": alias[1]} for alias in cursor])

In [ ]:
t_df = df_aliases.filter(df_aliases["count"] < 80)
px.histogram(t_df, x="count", nbins=200)

In [ ]:
cursor = pw_mapping.database.execute_sql("select source_item_id, count(source_item_id) from statements group by source_item_id limit 1000000")
df_sources = pd.DataFrame([{"item_id": source[0], "count": source[1]} for source in cursor])

In [ ]:
t_df = df_sources.filter(df_sources["count"] < 250)
px.histogram(t_df, x="count", nbins=200)

In [ ]:
cursor = pw_mapping.database.execute_sql(
    "select target_item_id, count(target_item_id) from statements group by target_item_id limit 1000000")
df_targets = pd.DataFrame([{"item_id": target[0], "count": target[1]} for target in cursor])

In [ ]:
t_df = df_targets.filter(df_targets["count"] < 250)
px.histogram(t_df, x="count", nbins=200)

In [ ]:
cursor = pw_mapping.database.execute_sql(
    "select edge_property_id, count(edge_property_id) from statements group by edge_property_id limit 1000000")
df_props = pd.DataFrame([{"item_id": prop[0], "count": prop[1]} for prop in cursor])

In [ ]:
t_df = df_props.filter(df_props["count"] < 10000)
px.histogram(t_df, x="count", nbins=200)

In [ ]:
df_joined = df_pages.join(df_aliases, on="item_id", how="left")
df_joined = df_joined.drop_nulls()
df_joined

In [ ]:
t_df = df_joined.filter(df_joined["views"] < 400)
px.scatter(t_df, x="views", y="count")

In [ ]:
cursor = pw_mapping.database.execute_sql(
    """
    SELECT
  "source"."edge_property_id" AS "edge_property_id",
  "source"."count" AS "count",
  "Property - Edge Property"."property_id" AS "Property - Edge Property__property_id",
  "Property - Edge Property"."en_label" AS "Property - Edge Property__en_label",
  "Property - Edge Property"."en_description" AS "Property - Edge Property__en_description"
FROM
  (
    SELECT
      "public"."statements"."edge_property_id" AS "edge_property_id",
      COUNT(*) AS "count"
    FROM
      "public"."statements"

GROUP BY
      "public"."statements"."edge_property_id"

ORDER BY
      "public"."statements"."edge_property_id" ASC
  ) AS "source"

LEFT JOIN "public"."property" AS "Property - Edge Property" ON "source"."edge_property_id" = "Property - Edge Property"."property_id"
ORDER BY
  "source"."count" DESC
LIMIT
  20"""
)
df_props = pd.DataFrame([{"prop": prop[0], "count": prop[1], "label": prop[3]} for prop in cursor])

In [ ]:
cursor = pw_mapping.database.execute_sql(
    """SELECT
  "public"."page"."page_id" AS "page_id",
  "public"."page"."item_id" AS "item_id",
  "public"."page"."title" AS "title",
  "public"."page"."views" AS "views",
  "augmented statements - Item"."source_item_id" AS "augmented statements - Item__source_item_id",
  "augmented statements - Item"."edge_property_id" AS "augmented statements - Item__edge_property_id",
  "augmented statements - Item"."target_item_id" AS "augmented statements - Item__target_item_id",
  "augmented statements - Item"."prop counts - Edge Property__edge_property_id" AS "augmented statements - Item__prop counts - Edge Pro_66e87521",
  "augmented statements - Item"."prop counts - Edge Property__count" AS "augmented statements - Item__prop counts - Edge Pro_77fde572",
  "augmented statements - Item"."prop counts - Edge Property__Property - Edge Proper_fdc36d11" AS "augmented statements - Item__prop counts - Edge Pro_06760e39",
  "augmented statements - Item"."prop counts - Edge Property__Property - Edge Proper_e382230b" AS "augmented statements - Item__prop counts - Edge Pro_384a910a"
FROM
  "public"."page"

LEFT JOIN (
    SELECT
      "public"."statements"."source_item_id" AS "source_item_id",
      "public"."statements"."edge_property_id" AS "edge_property_id",
      "public"."statements"."target_item_id" AS "target_item_id",
      "prop counts - Edge Property"."edge_property_id" AS "prop counts - Edge Property__edge_property_id",
      "prop counts - Edge Property"."count" AS "prop counts - Edge Property__count",
      "prop counts - Edge Property"."Property - Edge Property__property_id" AS "prop counts - Edge Property__Property - Edge Proper_fdc36d11",
      "prop counts - Edge Property"."Property - Edge Property__en_label" AS "prop counts - Edge Property__Property - Edge Proper_e382230b"
    FROM
      "public"."statements"
      LEFT JOIN (
        SELECT
          "source"."edge_property_id" AS "edge_property_id",
          "source"."count" AS "count",
          "Property - Edge Property"."property_id" AS "Property - Edge Property__property_id",
          "Property - Edge Property"."en_label" AS "Property - Edge Property__en_label"
        FROM
          (
            SELECT
              "public"."statements"."edge_property_id" AS "edge_property_id",
              COUNT(*) AS "count"
            FROM
              "public"."statements"

GROUP BY
              "public"."statements"."edge_property_id"

ORDER BY
              "public"."statements"."edge_property_id" ASC
          ) AS "source"
          LEFT JOIN "public"."property" AS "Property - Edge Property" ON "source"."edge_property_id" = "Property - Edge Property"."property_id"
        ORDER BY
          "source"."count" DESC

LIMIT
          20
      ) AS "prop counts - Edge Property" ON "public"."statements"."edge_property_id" = "prop counts - Edge Property"."Property - Edge Property__property_id"
  ) AS "augmented statements - Item" ON "public"."page"."item_id" = "augmented statements - Item"."source_item_id"
"""
)
df_pages = pd.DataFrame([{"page_id": page[0], "views": page[3], "item": page[4]} for page in cursor])
df_pages = df_pages.group_by("page_id", "views", "item").all()
t_df = df_pages.map_rows(lambda row: (row[0],row[1][0],row[2][0]))
df_pages_sources = pd.DataFrame((t_df["column_0"].alias("page_id"), t_df["column_1"].alias("views"), t_df["column_2"].alias("item")))